## Advent of Code - Day 8

In [5]:
import tempfile
from contextlib import contextmanager

In [6]:
@contextmanager
def test_file(test_input):
    with tempfile.NamedTemporaryFile('r+') as f:
        f.write(test_input)
        f.seek(0)
        yield f

### Part 1

Build a tree from the input and then traverse the tree to compute the sum. Addition is associative and commutative so it doesn't really matter how we traverse the tree. Begin by defining a node in the tree:

In [170]:
class Node:
    def __init__(self, metadata, children):
        self.metadata = metadata
        self.children = children

and a function to read each number from the input file:

In [171]:
def get_header(path):
    """Returns an iterator for the header file at path."""
    with open(path, 'r') as f:
        num = []
        while True:
            data = f.read(1024)  # read 1KB chunks
            if not data:
                break
            for c in data:
                if c.isspace():
                    yield int(''.join(num))
                    num = []
                else:
                    num.append(c)
        yield int(''.join(num))

In [172]:
test_input = """2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2"""

In [173]:
with test_file(test_input) as f:
    for num in get_header(f.name):
        print('%d' % num, end=' ')
    print()

2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2 


Now to recursively build the tree and reduce it by summing the metadata entries:

In [174]:
def build_tree(header_iter):
    n_children = next(header_iter)
    n_metadata = next(header_iter)
    
    children = []
    for _ in range(n_children):
        children.append(build_tree(header_iter))
        
    metadata = []
    for _ in range(n_metadata):
        metadata.append(next(header_iter))
        
    return Node(metadata, children)

In [196]:
def reduce(root):
    """Reduces tree by summing all metadata entries."""
    val = sum(root.metadata)
    for child in root.children:
        val += reduce(child)
    return val

Try on test input:

In [197]:
with test_file(test_input) as f:
    tree = build_tree(get_header(f.name))
    print(reduce(tree))

138


Now on real input:

In [198]:
tree = build_tree(get_header('input'))
reduce(tree)

45618

### Part 2

In [227]:
def reduce(root):
    if len(root.children) == 0:
        return sum(root.metadata)
    
    val = 0
    for i in root.metadata:
        if i == 0:
            continue
        try:
            val += reduce(root.children[i - 1])
        except IndexError:
            continue
    return val

Try on test input:

In [228]:
with test_file(test_input) as f:
    tree = build_tree(get_header(f.name))
    print(reduce(tree))

66


and on real input:

In [229]:
tree = build_tree(get_header('input'))
reduce(tree)

22306